In [1]:
import os

import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
from scipy.integrate import simpson
from tqdm.notebook import tqdm

from lib import merge_in_geometry

/tmp/ipykernel_2451494/1907831832.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [3]:
plt.style.use('dark_background')  # for cool points

In [4]:
years = list(range(2017, 2023))  # years we have POUS data for
outage_threshold = 0.2  # OutageFraction above this is considered an outage
resample_freq = "1D"  # resample raw hourly data to this resolution, then check for outage state
start_buffer = "2D"  # when plotting outage timeseries, start this delta ahead of first outage period
end_buffer = "1W"  # when plotting outage timeseries, end this delta after last outage period

root_dir = "data"
states = pd.read_csv(
    os.path.join(
        root_dir,
        "raw",
        "states",
        "state_codes.csv"
    )
).set_index("state_fips_code")
county_boundaries: gpd.GeoDataFrame = gpd.read_file(
    os.path.join(
        root_dir,
        "raw",
        "counties",
        "cb_2018_us_county_500k.shp"
    )
)
outage_integrals_path = os.path.join(
    "data",
    "processed",
    "outage",
    f"{resample_freq}_county_outage_integrals.csv"
)

In [5]:
# read source data
data_by_year = {}
for year in years:
    processed_path = os.path.join(root_dir, f"processed/outage/{year}.parquet")
    data = pd.read_parquet(processed_path)
    data.OutageFraction = np.clip(data.OutageFraction, 0, 1)
    data_by_year[year] = data

# find set of all counties in data
counties = set()
for year, data in data_by_year.items():
    counties = counties | set(data.index.get_level_values("CountyFIPS"))
counties = sorted(counties)

# another view of source data, concat into single dataframe
all_counties_hourly = pd.concat(data_by_year).drop(columns=["CustomersTracked", "CustomersOut"])
all_counties_hourly = all_counties_hourly.droplevel(0)

In [6]:
# construct complete timeseries
# resample to resample_freq and take mean of OutageFraction
# save to disk as cache

if os.path.exists(outage_integrals_path):
    df = pd.read_csv(outage_integrals_path, dtype={"CountyFIPS": str})    
    
else:
    resampled_data_by_year = []
    for county_code in tqdm(counties):

        # whole timeseries April-October for single county
        for year in years:
            df = data_by_year[year]
            try:
                data = df.loc(axis=0)[:, county_code].reset_index(level="CountyFIPS")
                complete_index = pd.date_range(f"{year}-04-01", f"{year}-10-31", freq="1H")
                data = data.reindex(index=complete_index, fill_value=0)
                data.index.name = "RecordDateTime"
            except KeyError:
                continue

            data = data.drop(columns=["CountyFIPS"]).resample(resample_freq).mean()
            data["CountyFIPS"] = county_code
            resampled_data_by_year.append(data)
            
    df = pd.concat(resampled_data_by_year)
    df.to_csv(outage_integrals_path)

In [7]:
plot_dir = os.path.join(
    "plots",
    "outage_timeseries_county",
    f"resample_{resample_freq}",
    f"threshold_{outage_threshold}"
)
os.makedirs(plot_dir, exist_ok=True)

# take the resampled data and filter to periods with OutageFraction above a threshold
outages = df.set_index(pd.to_datetime(df.RecordDateTime)).drop("RecordDateTime", axis=1)
outages = outages.drop(["CustomersTracked", "CustomersOut"], axis=1)
outages = outages[outages.OutageFraction > outage_threshold]

# duration of single resampling period in nanoseconds
length_of_resample_period_ns = pd.Timedelta(resample_freq).total_seconds() * 1E9

for county_code in tqdm(set(outages.CountyFIPS)):
    
    county_outages_resampled: pd.DataFrame = outages[outages.CountyFIPS == county_code]
    county_data_hourly: pd.DataFrame = all_counties_hourly.loc[(slice(None), county_code), :]
    
    try:
        county_admin_data: pd.Series = county_boundaries.sort_values("GEOID").set_index("GEOID").loc[county_code, :]
        state_code: str = county_admin_data.STATEFP
        state_name: str = states.loc[int(state_code), "state_name"]
        county_name: str = county_admin_data.NAME
    except Exception as e:
        state_name = "-"
        county_name = "-"

    # picking out runs of resampled outage periods
    start = 0
    outage_period_resampled_indicies: list[tuple[int, int]] = []
    for i, time_gap_ns in enumerate(np.diff(county_outages_resampled.index.values)):
        # should probably check that we are more than a fraction of a nanosecond different
        # 25 hour days etc.
        if float(time_gap_ns) != length_of_resample_period_ns:
            outage_period_resampled_indicies.append((start, i))
            start = i + 1

    for period_indicies in outage_period_resampled_indicies:
        
        start_i, end_i = period_indicies
        n_periods: int = end_i - start_i
        
        # check outage is at least a day long
        if (pd.Timedelta(resample_freq) * n_periods) > pd.Timedelta("1D"):
            
            # retrieve indicies of resampled run of outage periods
            group_datetimeindex: pd.DatetimeIndex = county_outages_resampled.iloc[start_i: end_i + 1].index
            
            # add a buffer around the start and end of the run
            start: str = str((group_datetimeindex[0] - pd.Timedelta(start_buffer)).date())
            end: str = str((group_datetimeindex[-1] + pd.Timedelta(end_buffer)).date())
            
            f, ax = plt.subplots(figsize=(9, 6))
            
            # select our hourly data to plot
            county_data_hourly.droplevel(1).loc[start: end, "OutageFraction"].plot(
                ax=ax,
                x_compat=True  # enforce standard matplotlib date tick labelling "2023-09-21"
            )

            ax.set_title(f"{county_name} ({county_code}), {state_name}")
            ax.set_ylabel("OutageFraction", labelpad=20)
            ax.set_xlabel("Time", labelpad=20)
            ax.set_ylim(0, 1.1)
            ax.grid(alpha=0.3)
            plt.subplots_adjust(bottom=0.2, top=0.9, left=0.1, right=0.91)

            f.savefig(
                os.path.join(
                    plot_dir,
                    f"{start}_{end}_{county_code}_{county_name.replace(' ', '_')}_{state_name}.png"
                )
            )
            plt.close(f)

  0%|          | 0/1307 [00:00<?, ?it/s]